# Task2 
## Modelo Oculto de Markov

In [11]:
# clase de HMM
import random
class HMM:
    def __init__(self,states,obervations,inital_prob,transition_prob,emission_prob):
        self.states = states
        self.observations = obervations
        self.initial_prob = inital_prob
        self.transition_prob = transition_prob
        self.emission_prob = emission_prob
    def generate_sequence(self,lengths):
        sequence = []
        # Generate subsequent observations using the Viterbi algorithm
        current_state = random.choices(self.states, weights=[self.initial_prob[state] for state in self.states])[0]
        for _ in range(1, lengths+1):
            next_state = random.choices(self.states, weights=[self.transition_prob[current_state][state] for state in self.states])[0]
            sequence.append(next_state)
            current_state = next_state
        return sequence
    
    def forward(self,observations):
        forward_prob = [{}]
        # Initialize base case
        for state in self.states:
            forward_prob[0][state] = self.initial_prob[state] * self.emission_prob[state][observations[0]]
        
        # Forward algorithm recursion
        for t in range(1, len(observations)):
            forward_prob.append({})
            for state_to in self.states:
                forward_prob[t][state_to] = sum(forward_prob[t-1][state_from] * self.transition_prob[state_from][state_to] * self.emission_prob[state_to][observations[t]] for state_from in self.states)
        
        return forward_prob
    
    def backward(self,observations):
        backward_prob = [{} for _ in range(len(observations))]
        # Initialize base case
        for state in self.states:
            backward_prob[-1][state] = 1
        
        # Backward algorithm recursion
        for t in range(len(observations)-2, -1, -1):
            for state_from in self.states:
                backward_prob[t][state_from] = sum(self.transition_prob[state_from][state_to] * self.emission_prob[state_to][observations[t+1]] * backward_prob[t+1][state_to] for state_to in self.states)
        
        return backward_prob
    
    def compute_state_probabilities(self,observations):
        forward_prob = self.forward(observations)
        backward_prob = self.backward(observations)
        
        state_probabilities = []
        for t in range(len(observations)):
            state_prob = {}
            for state in self.states:
                state_prob[state] = (forward_prob[t][state] * backward_prob[t][state]) / sum(forward_prob[t][s] * backward_prob[t][s] for s in self.states)
            state_probabilities.append(state_prob)
        
        return state_probabilities
    

In [10]:

# Uso y datos
states = ['Sunny', 'Rainy']
observations = ['Sunny', 'Sunny', 'Rainy'] 
initial_prob = {'Sunny': 0.5, 'Rainy': 0.5}
transition_prob = {"Sunny": {"Sunny": 0.8, 'Rainy': 0.2}, 'Rainy': {"Sunny": 0.4, "Rainy": 0.6}} 
emission_prob = {'Sunny': {'Sunny': 0.8, 'Rainy': 0.2}, 'Rainy': {'Sunny': 0.3, 'Rainy': 0.7}}
hmm = HMM(states, observations, initial_prob, transition_prob, emission_prob)
# Generar una secuencia de observaciones. 
obs_sequence = hmm.generate_sequence(5) 
print("Secuencia Generada:", obs_sequence)
# Calculo de probabilidades forward 
forward_probs = hmm.forward (observations) 
print("\nProbabilidades Forward:") 
print(forward_probs)
# Calculo de probabilidades backward 
backward_probs = hmm.backward (observations) 
print("\nProbabilidades Backward:") 
print(backward_probs)
# Calcular probabilidades de estado
state_probs = hmm.compute_state_probabilities (observations) 
print("\nProbabilidades de Estados:")
print(state_probs)

Secuencia Generada: ['Sunny', 'Rainy', 'Rainy', 'Rainy']

Probabilidades Forward:
[{'Sunny': 0.4, 'Rainy': 0.15}, {'Sunny': 0.30400000000000005, 'Rainy': 0.051000000000000004}, {'Sunny': 0.05272000000000002, 'Rainy': 0.06398000000000001}]

Probabilidades Backward:
[{'Sunny': 0.22200000000000006, 'Rainy': 0.18600000000000003}, {'Sunny': 0.30000000000000004, 'Rainy': 0.5}, {'Sunny': 1, 'Rainy': 1}]

Probabilidades de Estados:
[{'Sunny': 0.7609254498714653, 'Rainy': 0.23907455012853465}, {'Sunny': 0.7814910025706941, 'Rainy': 0.21850899742930588}, {'Sunny': 0.45175664095972584, 'Rainy': 0.5482433590402742}]
